In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Sample data
df = pd.read_csv(f'CS361_CourseDev_Trend\data_total.csv')
# fill NaN values with 0
df = df.fillna(0)

# Convert to PyTorch tensors
# X = torch.tensor(df.loc[:, 'HW1':'HW10'].values, dtype=torch.float32).view(-1, 10, 1)
# split data into train and test
X = torch.tensor(df.loc[:int(len(df)*0.8), 'HW1':'HW10'].values, dtype=torch.float32).view(-1, 10, 1)
X_test = torch.tensor(df.loc[int(len(df)*0.8):, 'HW1':'HW10'].values, dtype=torch.float32).view(-1, 10, 1)
y = torch.tensor(df.loc[:int(len(df)*0.8), 'mid1'].values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(df.loc[int(len(df)*0.8):, 'mid1'].values, dtype=torch.float32).view(-1, 1)


In [2]:
X.shape, y.shape, X_test.shape, y_test.shape

(torch.Size([273, 10, 1]),
 torch.Size([273, 1]),
 torch.Size([68, 10, 1]),
 torch.Size([68, 1]))

In [3]:

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        final_output = self.linear(lstm_out[:, -1, :])
        return final_output

# Hyperparameters
input_dim = 1
hidden_dim = 50
output_dim = 1
num_epochs = 100000
learning_rate = 0.01

model = LSTMModel(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    
    # Calculate accuracy
    with torch.no_grad():
        accuracy = torch.mean((torch.abs(outputs - y) <= 2.5).float()).item() * 100

    # Print loss and accuracy
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")

    # culculate accuracy for test for every 1000 epochs
    if (epoch+1) % 1000 == 0:
        with torch.no_grad():
            outputs_test = model(X_test)
            accuracy_test = torch.mean((torch.abs(outputs_test - y[int(len(df)*0.8):]) <= 2.5).float()).item() * 100
            print(f"Test Accuracy: {accuracy_test:.2f}%")

    # save model for every 10000 epochs named as LSTM_model_epoch_10000_Accuracy_XX.pt in models folder
    if (epoch+1) % 1000 == 0:
        torch.save(model.state_dict(), f'model\LSTM_model_epoch_{epoch+1}_Accuracy_{accuracy:.2f}_Accuracy_test{accuracy_test}.pt')

# Predict for a new student
# new_student_data = torch.tensor([[95, 93, ..., 92]], dtype=torch.float32).view(1, 10, 1)
# predicted_mid1 = model(new_student_data).item()
# print(f"Predicted mid1 score for the new student: {predicted_mid1:.2f}")


Epoch 10/100000, Loss: 13689.4805, Accuracy: 0.00%
Epoch 20/100000, Loss: 12273.0273, Accuracy: 0.00%
Epoch 30/100000, Loss: 11040.1074, Accuracy: 0.00%
Epoch 40/100000, Loss: 9895.9805, Accuracy: 0.00%
Epoch 50/100000, Loss: 8866.3770, Accuracy: 0.37%
Epoch 60/100000, Loss: 7938.6904, Accuracy: 0.00%
Epoch 70/100000, Loss: 7106.1738, Accuracy: 0.37%
Epoch 80/100000, Loss: 6358.2817, Accuracy: 0.37%
Epoch 90/100000, Loss: 5686.4780, Accuracy: 0.00%
Epoch 100/100000, Loss: 5083.1201, Accuracy: 0.73%
Epoch 110/100000, Loss: 4541.8477, Accuracy: 0.37%
Epoch 120/100000, Loss: 4057.1082, Accuracy: 0.37%
Epoch 130/100000, Loss: 3623.8997, Accuracy: 2.56%
Epoch 140/100000, Loss: 3237.4006, Accuracy: 2.93%
Epoch 150/100000, Loss: 2894.2441, Accuracy: 2.20%
Epoch 160/100000, Loss: 2590.3362, Accuracy: 1.47%
Epoch 170/100000, Loss: 2322.0200, Accuracy: 1.47%
Epoch 180/100000, Loss: 2085.9609, Accuracy: 2.93%
Epoch 190/100000, Loss: 1879.0410, Accuracy: 2.93%
Epoch 200/100000, Loss: 1698.3533, Ac

: 